In [ ]:
# connect to database at sqlite:///ipeds_data.db
import sqlite3
import pandas as pd
import os
import sys

# check if the database file exists
db_file = 'ipeds_data.db'
if not os.path.exists(db_file):
    print(f"Database file '{db_file}' does not exist.")
    sys.exit(1)


# connect to the database
conn = sqlite3.connect(db_file)
# create a cursor
cursor = conn.cursor()

In [ ]:


query = """
WITH CTE AS (
    SELECT
        enrl.unit_id
        ,inst.institution_name
        ,inst.state
        ,enrl.num_first_time_freshmen
        ,RANK() OVER (PARTITION BY inst.state ORDER BY enrl.num_first_time_freshmen DESC) AS rank
    FROM 
        enrollment enrl
        LEFT JOIN institution inst
        ON enrl.unit_id = inst.unit_id
)
SELECT *
FROM CTE 
WHERE rank = 1
;
"""

# run query
df = pd.read_sql_query(query, conn)
df.head()

,unit_id,institution_name,state,num_first_time_freshmen,rank
0,102553,University of Alaska Anchorage,AK,1306,1
1,100751,The University of Alabama,AL,8279,1
2,106397,University of Arkansas,AR,6337,1
3,104151,Arizona State University Campus Immersion,AZ,14102,1
4,110565,California State University-Fullerton,CA,7070,1


In [ ]:
# question: what were the most selective institutions
SELECT
    inst.unit_id
    ,inst.institution_name
    ,adm.num_applicants
    ,adm.num_admissions
    ,adm.num_admissions / adm.num_applicants AS adm_rate
FROM 
    institution inst 
LEFT JOIN admissions adm
ON inst.unit_id = adm.unit_id
WHERE adm.num_applicants > 1000
ORDER BY adm_rate 
LIMIT 100
;


In [ ]:
# visualization: charge and selectivity with color as considering workstudy